# Google Vertex

In this quickstart you will learn how to run evaluation functions using models from google Vertex like PaLM-2.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/models/google_vertex_quickstart.ipynb)

In [ ]:
#! pip install google-cloud-aiplatform==1.36.3 litellm==1.11.1 trulens_eval langchain==0.0.347

### Authentication

In [ ]:
from google.cloud import aiplatform

In [ ]:
aiplatform.init(
    project = "...",
    location="us-central1"
)

### Import from LangChain and TruLens

In [ ]:
# Imports main tools:
from trulens_eval import Feedback
from trulens_eval import LiteLLM
from trulens_eval import Tru
from trulens_eval import TruChain

tru = Tru()
tru.reset_database()


# Imports from langchain to build app. You may need to install langchain first
# with the following:
# ! pip install langchain>=0.0.170
from langchain.chains import LLMChain
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

### Create Simple LLM Application

This example uses a LangChain framework and OpenAI LLM

In [ ]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = VertexAI()

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

### Send your first request

In [ ]:
prompt_input = 'What is a good name for a store that sells colorful socks?'

In [ ]:
llm_response = chain(prompt_input)

display(llm_response)

## Initialize Feedback Function(s)

In [ ]:
# Initialize LiteLLM-based feedback function collection class:
litellm = LiteLLM(model_engine="chat-bison")

# Define a relevance function using LiteLLM
relevance = Feedback(litellm.relevance_with_cot_reasons).on_input_output()
# By default this will check relevance on the main app input and main app
# output.

## Instrument chain for logging with TruLens

In [ ]:
tru_recorder = TruChain(chain,
    app_id='Chain1_ChatApplication',
    feedbacks=[relevance])

In [ ]:
with tru_recorder as recording:
    llm_response = chain(prompt_input)

display(llm_response)

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0]

## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

## Or view results directly in your notebook

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all